In [50]:
import requests
from langchain_core.tools import tool
from langchain_groq import ChatGroq
from dotenv import load_dotenv

In [56]:
load_dotenv()

True

## Tool Creation

In [40]:
@tool
def weather_details(city:str)->float:
    """It gives the current weather of a place in celcius"""
    city_details = requests.get(f"https://geocoding-api.open-meteo.com/v1/search?name={city}").json()
    latitude = city_details['results'][0]['latitude']
    longitude = city_details['results'][0]['longitude']
    weather_data = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true").json()
    return float(weather_data['current_weather']['temperature'])

In [41]:
print(weather_details.name)
print(weather_details.args)
print(weather_details.description)

weather_details
{'city': {'title': 'City', 'type': 'string'}}
It gives the current weather of a place in celcius


In [44]:
weather_details.args_schema.model_json_schema()

{'description': 'It gives the current weather of a place in celcius',
 'properties': {'city': {'title': 'City', 'type': 'string'}},
 'required': ['city'],
 'title': 'weather_details',
 'type': 'object'}

In [45]:
weather_details.invoke({'city':'Udupi'})

28.6

In [47]:
weather_details.invoke({'city':'Bangalore'})

23.9

## Tool calling

In [57]:
def get_tools():
    return [weather_details]

In [59]:
llm = ChatGroq(model="llama-3.1-8b-instant")
llm_with_tools = llm.bind_tools(get_tools())

In [60]:
result = llm_with_tools.invoke("What is the temperature of Udupi")

In [61]:
result

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '1251s5d9d', 'function': {'arguments': '{"city":"Udupi"}', 'name': 'weather_details'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 232, 'total_tokens': 249, 'completion_time': 0.026935155, 'prompt_time': 0.014208387, 'queue_time': 0.049523592, 'total_time': 0.041143542}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_1151d4f23c', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--ed2d8dcc-7089-437f-b774-14463cc558a9-0', tool_calls=[{'name': 'weather_details', 'args': {'city': 'Udupi'}, 'id': '1251s5d9d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 232, 'output_tokens': 17, 'total_tokens': 249})

In [62]:
result.tool_calls

[{'name': 'weather_details',
  'args': {'city': 'Udupi'},
  'id': '1251s5d9d',
  'type': 'tool_call'}]

In [65]:
if(result.tool_calls[0]["name"] == "weather_details"):
   print(weather_details.invoke(result.tool_calls[0]["args"]))

28.6
